vars:
    - simulations:
        sim_a:
            filename: sim_a.pkl

stages:
    build_modules:
        foreach:
            sim_a:
                kwargs: sim_a.yaml
                params: sim_a_params.yaml
                dependancies: None
            gwf_a:
                kwargs: gwf.yaml
                params: gwf_params.yaml
                dependancies: sim_a.pkl
            dis:
                kwargs: dis.yaml
                params: dis_params.yaml
                dependancies: gwf_a.pkl
        do:
            cmd: python build_modules.py simulations.sim_a.filename ${key} ${item.kwargs}
            params: ${item.params}
            dependancies: ${item.dependancies}
            output: ${key}.pkl

# dis_a_params.yaml
# sample parameters file generated for dis_a

stress_period_data:
    elevation: dem.tif # timeseries input {0: dem.tif}
    conductivity: 1.0 # timeseries input {0: 1.0, 1: 2.0}

In [ ]:
#get expected defaults and datatypes from flopy
def inspect_class_defaults(cls_path:str) -> dict:
    import importlib
    import inspect
    
    try:
        # Try to import the class from the specified path
        module_name, class_name = cls_path.rsplit(".", 1)
        module = importlib.import_module(module_name)
        inspect_class = getattr(module, class_name)
        print(f"Inspecting {inspect_class.__name__} class")
    except ImportError as e:
        # Handle the case where the import fails
        raise ImportError(f"Could not import {cls_path}: {e}")

    signature = inspect.signature(inspect_class.__init__)

    defaults = {}
    for name, param in signature.parameters.items():
        if name != 'self':
            defaults[name] = param.default # can also checkout param.annotation or param.kind
    return defaults
# print("Expected types:")
# print(expected_types)

In [66]:
# example Module class
class Module:
    def __init__(
            self,
            modtype:str, # type of the module (ie. modflow, mt3d, etc)
            name:str=None, # name of the module
            sim_dependencies:dict=None, # dict of other module required (ie. dis) -> Module type
            param_dict=None, # dictionary of adjustable parameter names
            special_methods=None, # special methods to format params
            cmd=None, # command to run/build module
            **kwargs):
        self.modtype = modtype
        self.name = name
        self.sim_dependencies = sim_dependencies
        self.param_dict = param_dict
        self.special_methods = special_methods
        self.cmd = cmd

        # these will be the inputs to the cmd function when called
        self.cmd_kwargs = inspect_class_defaults(self.cmd)

        self.output = None

    def add_params(self, params: dict):
        # check if the key is in the cmd defaults
        # check if the key is in the special methods dict
        # update the cmd defaults with the new values
        for key, value in params.items():
            if key in self.cmd_kwargs.keys():
                
                # special logic for special methods
                if key in self.special_methods.keys():
                    # call special method
                    print(f'Calling special method for {key} with values {value}')
                else:
                    # set the parameter value
                    print(f'{key}:{value} is not a valid adjustable parameter for {self.name}')
    
    def run(self):
        # first run dependancies
        if self.sim_dependencies:
            self.run_dependancies()

        # run the command with the parameters
        print(f'Running {self.name}')
        print(f'Running {self.cmd} with parameters {self.cmd_kwargs}')
        # here you would call the cmd function with the cmd_kwargs as arguments
        # for example: self.cmd(**self.cmd_kwargs)
        self.output = f'{self.name} output'

    def run_dependancies(self):
        # run the dependancies first
        for cmd_kwarg, dep_module in self.sim_dependencies.items():
            # first check if previously ran
            if dep_module.output:
                print(f'{dep_module.name} already ran')
                self.sim_dependencies[cmd_kwarg] = dep_module.output # update the cmd_kwargs with the dependancy values
            else:
                # run the dependancy module
                dep_module.run()
                self.sim_dependencies[cmd_kwarg] = dep_module.output # update the cmd_kwargs with the dependancy values




In [67]:
sim = Module(
    modtype='sim',
    name = 'test_sim',
    sim_dependencies=None,
    param_dict={},
    special_methods=None,
    cmd='flopy.mf6.MFSimulation'
)

gwf = Module(
    modtype='gwf',
    name='test_gwf',
    sim_dependencies={'simulation': sim},
    param_dict={},
    special_methods=None,
    cmd='flopy.mf6.ModflowGwf'
)

# create a template drn module
drn = Module(
    modtype = 'drn',
    sim_dependencies={'model': gwf},
    param_dict={'stress_period_data': ['conductivity', 'elevation', 'mask']},
    cmd='flopy.mf6.ModflowGwfdrn',
    special_methods={'stress_period_data': 'get_stress_period_data function'},
)

# these would be inputs from the params file in dvc
params = {
    'stress_period_data': {
        'conductivity': 0.1,
        'elevation': 10.0,
        'mask': [1, 2, 3] # cellid
    },
    'save_flows': True,
}

# drn.add_params(params)
drn.run()

Inspecting MFSimulation class
Inspecting ModflowGwf class
Inspecting ModflowGwfdrn class
Running test_sim
Running flopy.mf6.MFSimulation with parameters {'sim_name': 'sim', 'version': 'mf6', 'exe_name': 'mf6', 'sim_ws': '.', 'verbosity_level': 1, 'write_headers': True, 'use_pandas': True, 'lazy_io': False, 'continue_': None, 'nocheck': None, 'memory_print_option': None, 'profile_option': None, 'maxerrors': None, 'print_input': None, 'hpc_data_data': None}
Running test_gwf
Running flopy.mf6.ModflowGwf with parameters {'simulation': <class 'inspect._empty'>, 'modelname': 'model', 'model_nam_file': None, 'version': 'mf6', 'exe_name': 'mf6', 'model_rel_path': '.', 'list': None, 'print_input': None, 'print_flows': None, 'save_flows': None, 'newtonoptions': None, 'nc_mesh2d_filerecord': None, 'nc_structured_filerecord': None, 'nc_filerecord': None, 'kwargs': <class 'inspect._empty'>}
Running None
Running flopy.mf6.ModflowGwfdrn with parameters {'model': <class 'inspect._empty'>, 'loading_pac

In [32]:
drn.cmd

'flopy.mf6.ModflowGwfdrn'

In [20]:
inspect_class_defaults('flopy.mf6.ModflowGwfdrn')

Inspecting ModflowGwfdrn class


{'model': inspect._empty,
 'loading_package': False,
 'auxiliary': None,
 'auxmultname': None,
 'auxdepthname': None,
 'boundnames': None,
 'print_input': None,
 'print_flows': None,
 'save_flows': None,
 'timeseries': None,
 'observations': None,
 'mover': None,
 'dev_cubic_scaling': None,
 'maxbound': None,
 'stress_period_data': None,
 'filename': None,
 'pname': None,
 'kwargs': inspect._empty}